# Data Loading, Transformation  and Analytics

This notebook loads and cleans BLS and Population data from S3, preparing them for analytics.


## Overview
- **Phase 1**: BLS Data Loading & Concatenation
- **Phase 2**: BLS Data Cleaning
- **Phase 3**: Population Data Loading
- **Phase 4**: Population Data Standardization
- **Phase 5**: Data Preparation & Cleaning
- **Phase 6**: Final Analytics (Excecuted tasks A,B,C)


## Setup & Configuration




In [69]:
import os
import json
import boto3
import pandas as pd
from dotenv import load_dotenv

# Load shared root .env file

current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

# Try multiple paths to find root .env file
env_paths = [
    os.path.join('..', '.env'),        
    os.path.join(os.path.dirname(current_dir), '.env'),  # Absolute path from current dir
    os.path.join('..', '..', '.env'),   
    '.env'                              #Final landing zone if rest all fail
]

env_loaded = False
for env_path in env_paths:
    abs_path = os.path.abspath(env_path)
    if os.path.exists(abs_path):
        load_dotenv(dotenv_path=abs_path, override=True)
        print(f"[OK] Loaded environment from: {abs_path}")
        env_loaded = True
        break
#Loading data from the .env
if not env_loaded:
    load_dotenv(override=True)
    print("[WARNING] Could not find .env file. Please ensure you're running from part3/ directory.")
    print("Expected location: ../.env (rearc_quest/.env)")


BUCKET = os.getenv('AWS_BUCKET_NAME')
PREFIX = os.getenv('AWS_BUCKET_PREFIX', '') or ''

if not BUCKET:
    raise ValueError("AWS_BUCKET_NAME not found in environment variables. Please check your .env file location.")

s3_client = boto3.client(
    's3',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    region_name=os.getenv('AWS_REGION')
)

print(f"S3 Bucket: {BUCKET}")
print(f"S3 Prefix: '{PREFIX}'")
print("\nSetup complete!")


Current working directory: c:\Users\suhru\Documents\rearc_quest\part3
[OK] Loaded environment from: c:\Users\suhru\Documents\rearc_quest\.env
S3 Bucket: rearc-bls
S3 Prefix: 'bls-data/'

Setup complete!


## Phase 1: BLS Data Loading & Concatenation


In [70]:
# Dynamically discover all BLS files in S3, and check with the prefix they are stored- here they are 'bls-data'
print(f"Listing BLS objects under prefix: '{PREFIX}' in bucket '{BUCKET}'")

paginator = s3_client.get_paginator('list_objects_v2')
bls_files = []

for page in paginator.paginate(Bucket=BUCKET, Prefix=PREFIX):
    for obj in page.get('Contents', []):
        key = obj['Key']
        # Just want to store the bls data so this is a sanity/validation check
        if key.endswith('/') or 'population_data.json' in key.lower():
            continue
        # One hint for this key assignment is that the files start with pr since it is from Puerto Rico so we will be adding that as a condition can update this if this was a mixed condition in the future.
        if 'pr.' in key:
            bls_files.append(key)

print(f"Found {len(bls_files)} BLS files:")
for f in bls_files[:5]:  
    print(f"  - {f}")
if len(bls_files) > 5:
    print(f" {len(bls_files) - 5} more")


Listing BLS objects under prefix: 'bls-data/' in bucket 'rearc-bls'
Found 12 BLS files:
  - bls-data/pr.class
  - bls-data/pr.contacts
  - bls-data/pr.data.0.Current
  - bls-data/pr.data.1.AllData
  - bls-data/pr.duration
 7 more


In [71]:

dfs_bls = []

for key in bls_files:
    try:
        obj = s3_client.get_object(Bucket=BUCKET, Key=key)
        body = obj['Body']
        
        # Seperator as one of the key conditions was whitespace and removing it is very necessary.
        df = pd.read_csv(body, sep=r'\s+', dtype=str, on_bad_lines='skip')
        df['source_key'] = key  
        dfs_bls.append(df)
        print(f"  Loaded {len(df)} rows")
    except Exception as e:
        print(f"   Failed to load {key}: {e}")

print(f"\nTotal BLS DataFrames collected: {len(dfs_bls)}")


  Loaded 1 rows
  Loaded 7 rows
  Loaded 37239 rows
  Loaded 75896 rows
  Loaded 3 rows
  Loaded 1 rows
  Loaded 2 rows
  Loaded 4 rows
  Loaded 1 rows
  Loaded 2 rows
  Loaded 282 rows
  Loaded 162 rows

Total BLS DataFrames collected: 12


In [72]:

if dfs_bls:
    df_bls_master = pd.concat(dfs_bls, ignore_index=True)
    print(f" Created df_bls_master with {len(df_bls_master):,} rows")
    print(f"Columns: {list(df_bls_master.columns)}")
    print(f"\nFirst few rows:")
    display(df_bls_master.head())
else:
    print(" No BLS files loaded!")


 Created df_bls_master with 113,600 rows
Columns: ['class_code', 'class_text', 'display_level', 'selectable', 'sort_sequence', 'source_key', 'Productivity', 'Contacts', 'series_id', 'year', 'period', 'value', 'footnote_codes', 'duration_code', 'duration_text', 'footnote_code', 'footnote_text', 'measure_code', 'measure_text', 'period_abbr', 'period_name', 'Seasonal_code', 'Seasonal_text', 'sector_code', 'sector_name', 'seasonal', 'base_year', 'begin_year', 'begin_period', 'end_year', 'end_period', 'Major', 'Sector', 'and', 'Costs', '(PR)']

First few rows:


,class_code,class_text,display_level,selectable,sort_sequence,source_key,Productivity,Contacts,series_id,year,...,base_year,begin_year,begin_period,end_year,end_period,Major,Sector,and,Costs,(PR)
0,3,Employees,0,T,2,bls-data/pr.class,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,bls-data/pr.contacts,(202),691-5606,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,bls-data/pr.contacts,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,bls-data/pr.contacts,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,bls-data/pr.contacts,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Phase 2: BLS Data Cleaning


In [73]:
# Main cleaning condition trim the white spaces that are present.
df_bls_master.columns = df_bls_master.columns.str.strip()
print("[OK] Trimmed column names")
print(f"Columns: {list(df_bls_master.columns)}")


[OK] Trimmed column names
Columns: ['class_code', 'class_text', 'display_level', 'selectable', 'sort_sequence', 'source_key', 'Productivity', 'Contacts', 'series_id', 'year', 'period', 'value', 'footnote_codes', 'duration_code', 'duration_text', 'footnote_code', 'footnote_text', 'measure_code', 'measure_text', 'period_abbr', 'period_name', 'Seasonal_code', 'Seasonal_text', 'sector_code', 'sector_name', 'seasonal', 'base_year', 'begin_year', 'begin_period', 'end_year', 'end_period', 'Major', 'Sector', 'and', 'Costs', '(PR)']


In [74]:
# 2. Trim whitespace in key text columns
text_columns = ['series_id', 'period']
for col in text_columns:
    if col in df_bls_master.columns:
        df_bls_master[col] = df_bls_master[col].astype(str).str.strip()
        print(f" Trimmed whitespace in '{col}'")
    else:
        print(f" Column '{col}' not found (skipping)")


 Trimmed whitespace in 'series_id'
 Trimmed whitespace in 'period'


In [75]:
# 3. Enforce numeric type for 'value' column
if 'value' in df_bls_master.columns:
    df_bls_master['value'] = pd.to_numeric(df_bls_master['value'], errors='coerce')
    print(f" Converted 'value' to numeric")
    print(f"  Non-numeric values converted to NaN: {df_bls_master['value'].isna().sum()}")
else:
    print("Column 'value' not found")


 Converted 'value' to numeric
  Non-numeric values converted to NaN: 465


In [76]:
# 4. Standardize 'year' column as integer
if 'year' in df_bls_master.columns:
    df_bls_master['year'] = pd.to_numeric(df_bls_master['year'], errors='coerce').astype('Int64')
    print(f" Converted 'year' to integer")
    print(f"  Year range: {df_bls_master['year'].min()} - {df_bls_master['year'].max()}")
else:
    print(" Column 'year' not found")

print("\nBLS Master DataFrame info:")
print(df_bls_master.info())


 Converted 'year' to integer
  Year range: 1947 - 2025

BLS Master DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113600 entries, 0 to 113599
Data columns (total 36 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   class_code      283 non-null     object 
 1   class_text      1 non-null       object 
 2   display_level   7 non-null       object 
 3   selectable      6 non-null       object 
 4   sort_sequence   6 non-null       object 
 5   source_key      113600 non-null  object 
 6   Productivity    121 non-null     object 
 7   Contacts        1 non-null       object 
 8   series_id       113600 non-null  object 
 9   year            113135 non-null  Int64  
 10  period          113600 non-null  object 
 11  value           113135 non-null  float64
 12  footnote_codes  656 non-null     object 
 13  duration_code   284 non-null     object 
 14  duration_text   2 non-null       object 
 15  footnote_code   1 n

## Phase 3: Population Data Loading


In [77]:
# Load population_data.json from S3, since there is no prefix in storage load directly from bucket.
pop_key = 'population_data.json'
print(f"Loading {pop_key} from bucket {BUCKET}...")

try:
    obj = s3_client.get_object(Bucket=BUCKET, Key=pop_key)
    raw = obj['Body'].read()
    payload = json.loads(raw)
    
    records = payload.get('data', [])
    if not records:
        raise ValueError("Population JSON has no 'data' records")
    
    df_pop = pd.DataFrame(records)
    print(f"Loaded {len(df_pop)} population records")
    print(f"Columns: {list(df_pop.columns)}")
    display(df_pop.head())
except Exception as e:
    print(f"Failed to load population data: {e}")
    print(f"The key which failed: {pop_key}")


Loading population_data.json from bucket rearc-bls...
Loaded 10 population records
Columns: ['Nation', 'Nation ID', 'Population', 'Year']


,Nation,Nation ID,Population,Year
0,United States,01000US,316128839.0,2013
1,United States,01000US,318857056.0,2014
2,United States,01000US,321418821.0,2015
3,United States,01000US,323127515.0,2016
4,United States,01000US,325719178.0,2017


## Phase 4: Population Data Standardization


In [78]:
#Standarizing to get all of them into the same page.
rename_map = {}
if 'Year' in df_pop.columns:
    rename_map['Year'] = 'year'
if 'Population' in df_pop.columns:
    rename_map['Population'] = 'population'

if rename_map:
    df_pop = df_pop.rename(columns=rename_map)
    print(f" Renamed columns: {rename_map}")
else:
    print("No columns to rename")


 Renamed columns: {'Year': 'year', 'Population': 'population'}


In [79]:
# Enforce integer year and numeric population
if 'year' in df_pop.columns:
    df_pop['year'] = pd.to_numeric(df_pop['year'], errors='coerce').astype('Int64')
    print(f"  Year range: {df_pop['year'].min()} - {df_pop['year'].max()}")
else:
    print("Column 'year' not found")

if 'population' in df_pop.columns:
    df_pop['population'] = pd.to_numeric(df_pop['population'], errors='coerce')
    print(f" Converted 'population' to numeric")
else:
    print(" Column 'population' not found")


  Year range: 2013 - 2023
 Converted 'population' to numeric


In [80]:
df_pop_clean = df_pop.copy()
print("\nPopulation DataFrame info:")
print(df_pop_clean.info())
print("\nFirst few rows:")
display(df_pop_clean.head(10))



Population DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Nation      10 non-null     object 
 1   Nation ID   10 non-null     object 
 2   population  10 non-null     float64
 3   year        10 non-null     Int64  
dtypes: Int64(1), float64(1), object(2)
memory usage: 458.0+ bytes
None

First few rows:


,Nation,Nation ID,population,year
0,United States,01000US,316128839.0,2013
1,United States,01000US,318857056.0,2014
2,United States,01000US,321418821.0,2015
3,United States,01000US,323127515.0,2016
4,United States,01000US,325719178.0,2017
5,United States,01000US,327167439.0,2018
6,United States,01000US,328239523.0,2019
7,United States,01000US,331893745.0,2021
8,United States,01000US,333287562.0,2022
9,United States,01000US,334914896.0,2023


## Summary & Ready for Analytics


In [81]:
print(f"\nBLS Master DataFrame:")
print(f"  Rows: {len(df_bls_master):,}")
print(f"  Columns: {list(df_bls_master.columns)}")
print(f"\nPopulation DataFrame:")
print(f"  Rows: {len(df_pop_clean):,}")
print(f"  Columns: {list(df_pop_clean.columns)}")
print("\n" + "=" * 60)
print("Part 1 and Part 2 joins are complete ready for further analytics")



BLS Master DataFrame:
  Rows: 113,600
  Columns: ['class_code', 'class_text', 'display_level', 'selectable', 'sort_sequence', 'source_key', 'Productivity', 'Contacts', 'series_id', 'year', 'period', 'value', 'footnote_codes', 'duration_code', 'duration_text', 'footnote_code', 'footnote_text', 'measure_code', 'measure_text', 'period_abbr', 'period_name', 'Seasonal_code', 'Seasonal_text', 'sector_code', 'sector_name', 'seasonal', 'base_year', 'begin_year', 'begin_period', 'end_year', 'end_period', 'Major', 'Sector', 'and', 'Costs', '(PR)']

Population DataFrame:
  Rows: 10
  Columns: ['Nation', 'Nation ID', 'population', 'year']

Part 1 and Part 2 joins are complete ready for further analytics


## Quick Join Example


In [82]:
# Trying out a quick join to see how well the code works till now and also a side task mentioned.
if 'year' in df_bls_master.columns and 'year' in df_pop_clean.columns:
    df_joined = df_bls_master.merge(
        df_pop_clean[['year', 'population']],
        on='year',
        how='left'
    )
    print(f"  Result: {len(df_joined):,} rows")
    print(f"  Rows with population data: {df_joined['population'].notna().sum():,}")
    display(df_joined.head())
else:
    print("[INFO] Cannot join - 'year' column missing in one or both DataFrames")


  Result: 113,600 rows
  Rows with population data: 24,420


,class_code,class_text,display_level,selectable,sort_sequence,source_key,Productivity,Contacts,series_id,year,...,begin_year,begin_period,end_year,end_period,Major,Sector,and,Costs,(PR),population
0,3,Employees,0,T,2,bls-data/pr.class,NaN,NaN,nan,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,bls-data/pr.contacts,(202),691-5606,nan,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,bls-data/pr.contacts,NaN,NaN,nan,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,bls-data/pr.contacts,NaN,NaN,nan,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,bls-data/pr.contacts,NaN,NaN,nan,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Phase 5: Final Data Preparation & Cleaning

Before analytics, ensure data is fully cleaned and consistent.


In [83]:
# Filter out metadata files - keep only time-series data files


print("=" * 60)
print("FILTERING: Time-Series Data Only")

if 'source_key' in df_bls_master.columns:
    # Count rows before filtering
    rows_before = len(df_bls_master)
    
    # Filter to only include rows from time-series data files
    # These files contain 'pr.data' in their source_key
    df_bls_master = df_bls_master[
        df_bls_master['source_key'].str.contains('pr.data', case=False, na=False)
    ].copy()
    
    rows_after = len(df_bls_master)
    rows_removed = rows_before - rows_after
    
    print(f"\nFiltering Results:")
    print(f"  Rows before: {rows_before:,}")
    print(f"  Rows after: {rows_after:,}")
    print(f"  Metadata rows removed: {rows_removed:,} ({rows_removed/rows_before*100:.1f}%)")
    print(f"\n[OK] Filtered to time-series data files only")
    print(f"     This optimizes Tasks B and C by removing metadata rows")
    
    # Show which source files remain
    if rows_after > 0:
        remaining_files = df_bls_master['source_key'].unique()
        print(f"\nRemaining source files ({len(remaining_files)}):")
        for f in sorted(remaining_files):
            print(f"  - {f}")
else:
    print("'source_key' column not found - cannot filter metadata files")
    print("Proceeding with full dataset (may include metadata rows)")



FILTERING: Time-Series Data Only

Filtering Results:
  Rows before: 113,600
  Rows after: 113,135
  Metadata rows removed: 465 (0.4%)

[OK] Filtered to time-series data files only
     This optimizes Tasks B and C by removing metadata rows

Remaining source files (2):
  - bls-data/pr.data.0.Current
  - bls-data/pr.data.1.AllData


## Phase 6: Analytical Tasks


In [84]:
#Final cleaning that is removing or handling the NaN values

print("FINAL DATA PREPARATION")


if 'value' in df_bls_master.columns:
    nan_count = df_bls_master['value'].isna().sum()
    total_count = len(df_bls_master)
    print(f"\nBLS Master DataFrame:")
    print(f"  Total rows: {total_count:,}")
    print(f"  NaN values in 'value': {nan_count:,} ({nan_count/total_count*100:.2f}%)")
    print(f"  Valid numeric values: {total_count - nan_count:,}")

    print(f" NaN values are expected from non-numeric entries (footnotes, dashes)")
    print(f"         They will be excluded from calculations automatically")

if 'year' in df_bls_master.columns:
    df_bls_master['year'] = df_bls_master['year'].astype('Int64')
    print(f"\n[OK] BLS 'year' column is integer type")

if 'year' in df_pop_clean.columns:
    df_pop_clean['year'] = df_pop_clean['year'].astype('Int64')
    print(f"[OK] Population 'year' column is integer type")

print(f"\nBLS columns: {list(df_bls_master.columns)}")
print(f"Population columns: {list(df_pop_clean.columns)}")


FINAL DATA PREPARATION

BLS Master DataFrame:
  Total rows: 113,135
  NaN values in 'value': 0 (0.00%)
  Valid numeric values: 113,135
 NaN values are expected from non-numeric entries (footnotes, dashes)
         They will be excluded from calculations automatically

[OK] BLS 'year' column is integer type
[OK] Population 'year' column is integer type

BLS columns: ['class_code', 'class_text', 'display_level', 'selectable', 'sort_sequence', 'source_key', 'Productivity', 'Contacts', 'series_id', 'year', 'period', 'value', 'footnote_codes', 'duration_code', 'duration_text', 'footnote_code', 'footnote_text', 'measure_code', 'measure_text', 'period_abbr', 'period_name', 'Seasonal_code', 'Seasonal_text', 'sector_code', 'sector_name', 'seasonal', 'base_year', 'begin_year', 'begin_period', 'end_year', 'end_period', 'Major', 'Sector', 'and', 'Costs', '(PR)']
Population columns: ['Nation', 'Nation ID', 'population', 'year']



### Data snapshot from 2013-2018 before proceeding.


In [85]:

print("FILTERING: Time-Series Data Only")


if 'source_key' in df_bls_master.columns:
    rows_before = len(df_bls_master)
    nan_before = df_bls_master['value'].isna().sum()
    
    df_bls_master = df_bls_master[
        df_bls_master['source_key'].str.contains('pr.data', case=False, na=False)
    ].copy()
    
    rows_after = len(df_bls_master)
    rows_removed = rows_before - rows_after
    nan_after = df_bls_master['value'].isna().sum()
    
    print(f"\nFiltering Results:")
    print(f"  Rows before: {rows_before:,}")
    print(f"  Rows after: {rows_after:,}")
    print(f"  Metadata rows removed: {rows_removed:,} ({rows_removed/rows_before*100:.1f}%)")
    print(f"\nNaN Impact:")
    print(f"  NaN values before: {nan_before:,}")
    print(f"  NaN values after: {nan_after:,}")
    print(f"  NaN reduction: {nan_before - nan_after:,}")
    print(f"\n[OK] Filtered to time-series data files only")
    print(f"     This optimizes Tasks B and C by removing metadata rows")
    
    if rows_after > 0:
        remaining_files = df_bls_master['source_key'].unique()
        print(f"\nRemaining source files ({len(remaining_files)}):")
        for f in sorted(remaining_files):
            file_rows = len(df_bls_master[df_bls_master['source_key'] == f])
            print(f"  - {f} ({file_rows:,} rows)")
else:
    print("[WARNING] 'source_key' column not found - cannot filter metadata files")
    print("          Proceeding with full dataset (may include metadata rows)")



FILTERING: Time-Series Data Only

Filtering Results:
  Rows before: 113,135
  Rows after: 113,135
  Metadata rows removed: 0 (0.0%)

NaN Impact:
  NaN values before: 0
  NaN values after: 0
  NaN reduction: 0

[OK] Filtered to time-series data files only
     This optimizes Tasks B and C by removing metadata rows

Remaining source files (2):
  - bls-data/pr.data.0.Current (37,239 rows)
  - bls-data/pr.data.1.AllData (75,896 rows)


### TASK A: Filter out years as mentioned and find std,mean

In [86]:

# Filter: Select rows where year is between 2013 and 2018 (inclusive)
# Calculate: Mean and standard deviation of population

print("TASK A: Population Statistics (2013-2018)")


df_pop_filtered = df_pop_clean[
    (df_pop_clean['year'] >= 2013) & 
    (df_pop_clean['year'] <= 2018)
].copy()

print(f"\nFiltered population data:")
print(f"  Years: 2013-2018")
print(f"  Rows: {len(df_pop_filtered)}")

if len(df_pop_filtered) > 0 and 'population' in df_pop_filtered.columns:
    # Calculate mean and standard deviation
    stats = df_pop_filtered['population'].agg(['mean', 'std'])
    
    print(f"\nResults:")
    print(f"  Mean Population: {stats['mean']:,.2f}")
    print(f"  Standard Deviation: {stats['std']:,.2f}")
    
    print(f"\nDetailed view:")
    display(df_pop_filtered[['year', 'population']])
    
    task_a_results = {
        'mean': stats['mean'],
        'std': stats['std'],
        'years': sorted(df_pop_filtered['year'].unique().tolist())
    }
else:
    print("[ERROR] No data found for years 2013-2018")
    task_a_results = None


TASK A: Population Statistics (2013-2018)

Filtered population data:
  Years: 2013-2018
  Rows: 6

Results:
  Mean Population: 322,069,808.00
  Standard Deviation: 4,158,441.04

Detailed view:


,year,population
0,2013,316128839.0
1,2014,318857056.0
2,2015,321418821.0
3,2016,323127515.0
4,2017,325719178.0
5,2018,327167439.0


### Task B: Best Year Report (Max Value per Series)


In [87]:
# Task B: Best Year Report
# Group by series_id and year, sum the values
# Find the year with maximum sum for each series_id

print("TASK B: Best Year Report (Max Value per Series)")

required_cols = ['series_id', 'year', 'value']
missing_cols = [col for col in required_cols if col not in df_bls_master.columns]
if missing_cols:
    print(f"[ERROR] Missing required columns: {missing_cols}")
else:
    # Step 1: Group by series_id and year, sum the values
    df_annual = df_bls_master.groupby(['series_id', 'year'], as_index=False)['value'].sum()
    
    print(f"\nAnnual totals (grouped by series_id and year):")
    print(f"  Total rows: {len(df_annual):,}")
    print(f"  Unique series: {df_annual['series_id'].nunique()}")
    display(df_annual.head(10))
    
    df_annual_sorted = df_annual.sort_values('value', ascending=False)
    
    # Step 3: Drop duplicates, keeping first (highest value) for each series_id
    df_best_years = df_annual_sorted.drop_duplicates(subset=['series_id'], keep='first')
    
    print(f"\nBest Year Results (one row per series_id):")
    print(f"  Total series: {len(df_best_years)}")
    print(f"\nTop 10 series by value:")
    display(df_best_years.head(10))
    
    # Store results
    task_b_results = df_best_years[['series_id', 'year', 'value']].copy()
    print(f"\n[OK] Task B complete - {len(task_b_results)} series with best years identified")


TASK B: Best Year Report (Max Value per Series)

Annual totals (grouped by series_id and year):
  Total rows: 16,589
  Unique series: 282


,series_id,year,value
0,PRS30006011,1988,8.9
1,PRS30006011,1989,2.9
2,PRS30006011,1990,-7.5
3,PRS30006011,1991,-17.4
4,PRS30006011,1992,-8.4
5,PRS30006011,1993,0.4
6,PRS30006011,1994,6.4
7,PRS30006011,1995,14.2
8,PRS30006011,1996,-1.0
9,PRS30006011,1997,8.8



Best Year Results (one row per series_id):
  Total series: 282

Top 10 series by value:


,series_id,year,value
16115,PRS88003183,2024,2128.410
16351,PRS88003193,2024,1721.676
13755,PRS88003083,2024,1601.586
10215,PRS85006083,2024,1557.384
6675,PRS84006083,2024,1557.054
13283,PRS88003053,2024,1531.108
9743,PRS85006053,2024,1502.368
6203,PRS84006053,2024,1501.740
13519,PRS88003063,2024,1481.352
9979,PRS85006063,2024,1460.022



[OK] Task B complete - 282 series with best years identified


### Task C: Unified Report (Specific Series & Period)


In [88]:
# Task C: Unified Report
# Merge BLS and Population data, then filter for specific series and period


print("TASK C: Unified Report (Series PRS30006032, Period Q01)")

# Step 1: Merge BLS master with Population data on year
if 'year' in df_bls_master.columns and 'year' in df_pop_clean.columns:
    df_merged = df_bls_master.merge(
        df_pop_clean[['year', 'population']],
        on='year',
        how='left'
    )
    
    print(f"\nMerged DataFrame:")
    print(f"  Total rows: {len(df_merged):,}")
    print(f"  Rows with population: {df_merged['population'].notna().sum():,}")
    
    # Step 2: Filter for specific series_id and period as mentioned in the task
    target_series = 'PRS30006032'
    target_period = 'Q01'
    
    df_filtered = df_merged[
        (df_merged['series_id'] == target_series) & 
        (df_merged['period'] == target_period)
    ].copy()
    
    print(f"\nFiltered for:")
    print(f"  series_id: {target_series}")
    print(f"  period: {target_period}")
    print(f"  Matching rows: {len(df_filtered)}")
    
    output_cols = ['series_id', 'year', 'period', 'value', 'population']
    available_cols = [col for col in output_cols if col in df_filtered.columns]
    
    if len(available_cols) == len(output_cols):
        task_c_results = df_filtered[output_cols].copy()
        
        print(f"\nFinal Report:")
        display(task_c_results)
        
        print(f"\n[OK] Task C complete - {len(task_c_results)} rows in unified report")
    else:
        missing = set(output_cols) - set(available_cols)
        print(f"[ERROR] Missing columns: {missing}")
        task_c_results = None
else:
    print("[ERROR] Cannot merge - 'year' column missing")
    task_c_results = None


TASK C: Unified Report (Series PRS30006032, Period Q01)

Merged DataFrame:
  Total rows: 113,135
  Rows with population: 24,420

Filtered for:
  series_id: PRS30006032
  period: Q01
  Matching rows: 69

Final Report:


,series_id,year,period,value,population
1064,PRS30006032,1995,Q01,0.0,NaN
1069,PRS30006032,1996,Q01,-4.2,NaN
1074,PRS30006032,1997,Q01,2.8,NaN
1079,PRS30006032,1998,Q01,0.9,NaN
1084,PRS30006032,1999,Q01,-4.1,NaN
...,...,...,...,...,...
38732,PRS30006032,2021,Q01,0.7,331893745.0
38737,PRS30006032,2022,Q01,5.3,333287562.0
38742,PRS30006032,2023,Q01,0.3,334914896.0
38747,PRS30006032,2024,Q01,-0.7,NaN



[OK] Task C complete - 69 rows in unified report


## Final Summary Report


In [89]:

print("PART 3: DATA ANALYTICS - FINAL SUMMARY")

print("\n TASK A: Population Statistics (2013-2018)")
if task_a_results:
    print(f"   Mean Population: {task_a_results['mean']:,.2f}")
    print(f"   Standard Deviation: {task_a_results['std']:,.2f}")
    print(f"   Years analyzed: {task_a_results['years']}")
else:
    print("   [ERROR] Task A failed")

print("\nTASK B: Best Year Report")
if 'task_b_results' in globals() and task_b_results is not None:
    print(f"   Total series analyzed: {len(task_b_results)}")
    print(f"   Sample results (top 5):")
    display(task_b_results.head())
else:
    print("   [ERROR] Task B failed")

print("\n TASK C: Unified Report (PRS30006032, Q01)")
if 'task_c_results' in globals() and task_c_results is not None:
    print(f"   Total rows: {len(task_c_results)}")
    print(f"   Columns: {list(task_c_results.columns)}")
    display(task_c_results)
else:
    print("   [ERROR] Task C failed")

print('End of analysis of merged data from BLS and population.json')

PART 3: DATA ANALYTICS - FINAL SUMMARY

 TASK A: Population Statistics (2013-2018)
   Mean Population: 322,069,808.00
   Standard Deviation: 4,158,441.04
   Years analyzed: [2013, 2014, 2015, 2016, 2017, 2018]

TASK B: Best Year Report
   Total series analyzed: 282
   Sample results (top 5):


,series_id,year,value
16115,PRS88003183,2024,2128.410
16351,PRS88003193,2024,1721.676
13755,PRS88003083,2024,1601.586
10215,PRS85006083,2024,1557.384
6675,PRS84006083,2024,1557.054



 TASK C: Unified Report (PRS30006032, Q01)
   Total rows: 69
   Columns: ['series_id', 'year', 'period', 'value', 'population']


,series_id,year,period,value,population
1064,PRS30006032,1995,Q01,0.0,NaN
1069,PRS30006032,1996,Q01,-4.2,NaN
1074,PRS30006032,1997,Q01,2.8,NaN
1079,PRS30006032,1998,Q01,0.9,NaN
1084,PRS30006032,1999,Q01,-4.1,NaN
...,...,...,...,...,...
38732,PRS30006032,2021,Q01,0.7,331893745.0
38737,PRS30006032,2022,Q01,5.3,333287562.0
38742,PRS30006032,2023,Q01,0.3,334914896.0
38747,PRS30006032,2024,Q01,-0.7,NaN


End of analysis of merged data from BLS and population.json
